In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pip install keras==2.1.6

     |████████████████████████████████| 348kB 2.7MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [5]:
#adapted from https://github.com/nxs5899/Named-Entity-Recognition_DeepLearning-keras

from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K

tags = ["I-Item", "B-Item", "I-Act",
        "B-Act", "I-Stat", "B-Stat", "O"]

max_len = 15
batch_size = 25
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]


input_text = Input(shape=(max_len,), dtype=tf.string)
embed = Lambda(ElmoEmbedding, output_shape=(None, 1024))(input_text)

x = LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.3, dropout=0.3)(embed)
x_rnn = LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.3, dropout=0.3)(x)

x = add([x, x_rnn])  # residual connection to the first biLSTM
out = TimeDistributed(Dense(len(tags), activation="softmax"))(x)

model = Model(input_text, out)


Using TensorFlow backend.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
#adapted from https://github.com/nxs5899/Named-Entity-Recognition_DeepLearning-keras

import numpy as np
import pandas as pd
import time

start = time.time()

file = 'drive/My Drive/Colab Notebooks/dat/train.txt'
separator = "~"
sentences = []


with open(file, 'r') as f:
    for num, line in enumerate(f):
        pairs = line.strip().split()
        sentence = []
        for pair in pairs:
            temp = pair.split(separator)
            temp.append(num)
            sentence.append(tuple(temp))
        sentences.append(sentence)


tag2idx = {t: i for i, t in enumerate(tags)}

X = [[w[0] for w in s] for s in sentences]

new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
X = new_X

y = [[tag2idx[w[1]] for w in s] for s in sentences]

from keras.preprocessing.sequence import pad_sequences

y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.05, random_state=2019)


import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K

sess = tf.Session()
K.set_session(sess)


sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())


model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


X_tr, X_val = X_tr[:1213*batch_size], X_tr[-135*batch_size:]
y_tr, y_val = y_tr[:1213*batch_size], y_tr[-135*batch_size:]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),
                    batch_size=batch_size, epochs=3, verbose=1)

hist = pd.DataFrame(history.history)

model.save_weights("model-5k-3-epoch.h5")
print("Saved model to disk")
end = time.time()
print(end - start)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3800 samples, validate on 3375 samples
Epoch 1/3



3775/3800 [============================>.] - ETA: 3s - loss: 0.2197 - acc: 0.9141

In [0]:
https://github.com/nxs5899/Named-Entity-Recognition_DeepLearning-keras

file = 'drive/My Drive/Colab Notebooks/dat/ExpertTest.txt'

separator = "~"
sentences = []


with open(file, 'r') as f:
    for num, line in enumerate(f):
        pairs = line.strip().split()
        sentence = []
        for pair in pairs:
            temp = pair.split(':')[-1].split(separator)
            temp.append(num)
            sentence.append(tuple(temp))
        sentences.append(sentence)


tag2idx = {t: i for i, t in enumerate(tags)}

X = [[w[0] for w in s] for s in sentences]

new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
X = new_X

y = [[tag2idx[w[1]] for w in s] for s in sentences]

from keras.preprocessing.sequence import pad_sequences

y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

from sklearn.model_selection import train_test_split

X_tmp, X_te, y_tmp, y_te = train_test_split(X, y, test_size=0.95, random_state=2019)

In [0]:
import numpy as np
import re
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

option = 1
test_file = 'drive/My Drive/Colab Notebooks/dat/ExpertTest.txt'

act = 0
item= 0
stat= 0
o=0
act_tag =0
item_tag=0
stat_tag=0
o_tag=0
act_count =0
item_count=0
stat_count=0
o_count=0

p_txt = 'drive/My Drive/Colab Notebooks/dat/kerasPred.txt'

with open(p_txt, 'w') as p_txt:
  if option == 1:
      accuracy = 0
      true_acc=0
      total = 0
      for i in range(len(X_te)):
          p = model.predict(np.array(X_te[i:i + batch_size]))[0]
          p = np.argmax(p, axis=-1)
          #print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
          #print("=" * 30)
          #print(i)
          if total != 0:
              #print(np.array(X_te[i:i + batch_size]))
              print(i, ' : ', accuracy / total * 100)
              print(act,item,stat,o)
              print(act_tag,item_tag,stat_tag,o_tag)
              print(act_count,item_count,stat_count,o_count)
          for w, true, pred in zip(X_te[i], y_te[i], p):
              if w != "__PAD__":
                  #keras.write("{:15}:{:5} ({})".format(w, tags[pred], tags[true]))
                  if tags[true] == tags[pred]:
                      if tags[true].endswith('Act'):
                        act = act+1
                      elif tags[true].endswith('Item'):
                        item = item+1                    
                      elif tags[true].endswith('Stat'):
                        stat = stat+1
                      elif tags[true].endswith('O'):
                        o = o+1

                      accuracy = accuracy + 1
                      tru_acc = true_acc + 1

                      # else if tags match but BIO tags are wrong, increment by half
                  elif tags[true].split('-')[-1] == tags[pred].split('-')[-1]:
                      if tags[true].endswith('Act'):
                        act = act+1
                      elif tags[true].endswith('Item'):
                        item = item+1                    
                      elif tags[true].endswith('Stat'):
                        stat = stat+1
                      elif tags[true].endswith('O'):
                        o = o+1
                      accuracy = accuracy + 0.5
                      tru_acc = true_acc + 1

                  total = total + 1
                  if tags[true].endswith('Act'):
                      act_count = act_count+1
                  elif tags[true].endswith('Item'):
                      item_count = item_count+1           
                  elif tags[true].endswith('Stat'):
                      stat_count = stat_count+1
                  elif tags[true].endswith('O'):
                      o_count = o_count+1
                      
                  if tags[pred].endswith('Act'):
                    act_tag = act_tag +1
                  elif tags[pred].endswith('Item'):
                    item_tag = item_tag +1
                  elif tags[pred].endswith('Stat'):
                    stat_tag = stat_tag +1
                  elif tags[pred].endswith('O'):
                    o_tag = o_tag +1

  else:
      i = 19
      p = model.predict(np.array(X_te[i:i + batch_size]))[0]
      p = np.argmax(p, axis=-1)
      print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
      print("=" * 30)
      for w, true, pred in zip(X_te[i], y_te[i], p):
          if w != "__PAD__":
              print("{:15}:{:5} ({})".format(w, tags[pred], tags[true]))
            
print('total: ', accuracy / total * 100)


1  :  0.0
0 0 0 0
0 2 0 3
0 1 4 0
2  :  21.428571428571427
0 2 0 0
0 4 0 3
0 3 4 0
3  :  27.77777777777778
1 2 0 0
1 4 0 4
1 4 4 0
4  :  46.42857142857143
2 3 0 2
2 6 0 6
2 5 5 2
5  :  44.73684210526316
2 3 0 4
3 8 0 8
2 5 5 7
6  :  47.72727272727273
3 4 0 4
4 9 0 9
3 7 5 7
7  :  48.0
4 5 0 4
6 10 0 9
4 9 5 7
8  :  51.724137931034484
4 7 1 4
6 12 1 10
4 12 6 7
9  :  54.285714285714285
5 9 2 4
7 14 2 12
5 14 9 7
10  :  58.97435897435898
6 11 2 5
8 16 2 13
6 16 9 8
11  :  57.77777777777777
6 11 2 8
9 16 2 18
7 17 9 12
12  :  59.61538461538461
7 12 2 11
11 17 2 22
8 19 9 16
13  :  57.89473684210527
7 12 2 13
12 19 2 24
8 19 9 21
14  :  60.0
8 14 2 13
13 21 2 24
9 21 9 21
15  :  61.53846153846154
9 14 2 16
14 22 2 27
10 21 9 25
16  :  62.5
10 14 2 20
15 24 2 31
11 21 9 31
17  :  67.07317073170732
10 14 2 30
15 24 2 41
11 21 9 41
18  :  67.05882352941175
11 14 3 30
16 24 3 42
12 22 10 41
19  :  65.0
11 15 3 31
16 25 3 46
13 25 10 42
20  :  65.42553191489363
11 17 4 31
16 27 4 47
13 28 11 42

InvalidArgumentError: ignored